In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import uniform_filter1d

# Llegir l'arxiu CSV
file_path = '/Users/laurabenitosandoval/Downloads/gaze_data.csv'  # Ruta de l'arxiu
data = pd.read_csv(file_path)

# Convertir els timestamps a segons
data['time_seconds'] = (data['device_time_stamp'] - data['device_time_stamp'].min()) / 1e6

# Filtrar dades en una finestra de 60 segons
start_time_60s = 1900  # Inici de la finestra
end_time_60s = start_time_60s + 60  # Fi de la finestra
time_filter_60s = (data['time_seconds'] >= start_time_60s) & (data['time_seconds'] <= end_time_60s)
filtered_data_60s = data[time_filter_60s].copy()

# Eliminar valors NaN de les coordenades Y
filtered_data_60s.dropna(subset=['left_gaze_point_on_display_area_y'], inplace=True)

# Calcular velocitat crua (diferència entre punts consecutius)
filtered_data_60s['velocity_raw'] = np.gradient(
    filtered_data_60s['left_gaze_point_on_display_area_y'],  # Coordenades Y suavitzades
    filtered_data_60s['time_seconds']
)

# Eliminar valors NaN en la velocitat
filtered_data_60s.dropna(subset=['velocity_raw'], inplace=True)

# Identificar moments en què es prem la tecla "up"
up_key_times = filtered_data_60s[filtered_data_60s['key'] == 'up']['time_seconds']

# Detectar moviments cap avall quan es prem "up"
filtered_data_60s['movement_down_during_up'] = np.where(
    (filtered_data_60s['velocity_raw'] > 0) &  # Velocitat positiva (cap avall)
    (filtered_data_60s['time_seconds'].isin(up_key_times)),  # Mentre es prem "up"
    1,  # Moviment cap avall detectat
    0   # No hi ha moviment cap avall
)

# Calcular velocitats locals per a aquests moviments cap avall
filtered_data_60s['local_displacement'] = np.abs(
    np.diff(filtered_data_60s['left_gaze_point_on_display_area_y'], prepend=0)
)
filtered_data_60s['local_time'] = np.diff(filtered_data_60s['time_seconds'], prepend=0)
filtered_data_60s['local_velocity'] = filtered_data_60s['local_displacement'] / filtered_data_60s['local_time']

# Aplicar el filtre de moviments cap avall
downward_movements = filtered_data_60s[filtered_data_60s['movement_down_during_up'] == 1]

# Validar si hi ha prou moviments cap avall
if not downward_movements.empty:
    velocity_threshold = np.median(downward_movements['local_velocity'])
else:
    velocity_threshold = np.nan
    print("No s'han detectat suficients moviments cap avall per calcular el llindar.")

# Debug: Imprimir el llindar
print(f"Llindar de velocitat per a la fase lenta: {velocity_threshold}")

# Excloure saccades (components ràpids) i aïllar la fase lenta
filtered_data_60s['slow_phase_velocity'] = np.where(
    (np.abs(filtered_data_60s['velocity_raw']) <= velocity_threshold) & (~np.isnan(velocity_threshold)),
    filtered_data_60s['velocity_raw'],
    np.nan
)

# Eliminar valors NaN de la fase lenta
filtered_data_60s.dropna(subset=['slow_phase_velocity'], inplace=True)

# Validar i calcular la fase lenta suavitzada amb una finestra d'1 segon
sampling_rate = len(filtered_data_60s) / (end_time_60s - start_time_60s)
window_size = max(1, min(int(sampling_rate), len(filtered_data_60s) - 1))  # Assegurar una finestra vàlida

if len(filtered_data_60s) > 1:  # Verificar que hi hagi suficients punts per suavitzar
    filtered_data_60s['feature_1'] = uniform_filter1d(
        filtered_data_60s['slow_phase_velocity'], size=window_size, mode='nearest'
    )
else:
    print("No hi ha suficients punts per calcular la fase lenta.")
    filtered_data_60s['feature_1'] = np.nan

# Representar gràficament els resultats
plt.figure(figsize=(16, 12))  # Incrementar la mida del gràfic

# Gràfic 1: Coordenades de la mirada
plt.subplot(3, 1, 1)
plt.plot(
    filtered_data_60s['time_seconds'],
    filtered_data_60s['left_gaze_point_on_display_area_y'],
    label='Gaze Y Coordinate (60s Window)',
    color='black', linewidth=2
)
plt.title(f'Gaze Point vs Time')
plt.xlabel('Time (seconds)')
plt.ylabel('Gaze Y Coordinate')
plt.ylim(0.3, 0.8)
plt.grid(alpha=0.3)
plt.legend()

# Gràfic 2: Velocitats (Slow phase OKN i raw velocity)
plt.subplot(3, 1, 2)
# Velocitat crua
plt.plot(
    filtered_data_60s['time_seconds'],
    filtered_data_60s['velocity_raw'],
    label='Raw Velocity',
    color='lightgray', alpha=0.8, linewidth=1
)
# Velocitat suavitzada de la fase lenta
plt.plot(
    filtered_data_60s['time_seconds'],
    filtered_data_60s['feature_1'],
    label='Feature 1: Smoothed Slow Phase Velocity (1s Window)',
    color='black', linewidth=2, linestyle='-'
)
plt.title(f'Slow Phase Velocity OKN')
plt.xlabel('Time (seconds)')
plt.ylabel('Velocity (a.u.)')
plt.ylim(-0.5, 0.5)  # Ajustar segons les dades
plt.grid(alpha=0.3)
plt.legend()

# Gràfic 3: Esdeveniments de tecles (sense "down")
plt.subplot(3, 1, 3)
key_colors = {'up': 'black', 'left': 'red', 'right': 'red'}  # Eliminem "down"
valid_keys = [key for key in key_colors if key in filtered_data_60s['key'].unique()]  # Només tecles amb dades
for key in valid_keys:
    key_times_60s = filtered_data_60s[filtered_data_60s['key'] == key]['time_seconds']
    plt.scatter(key_times_60s, [key] * len(key_times_60s), color=key_colors[key], label=f'Key: {key}', s=20)

plt.title(f'Key Press Events')
plt.xlabel('Time (seconds)')
plt.yticks(valid_keys)  # Mostrar només tecles vàlides
plt.grid(alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()
